In [1]:
import math
import numpy as np
import pandas as pd
import pprint
import itertools
from collections import deque
from copy import deepcopy

In [2]:
vendor_dist = [[0.0, 1.6, 0.6, 2.0, 1.7, 1.6, 1.5],
               [1.6, 0.0, 1.9, 3.5, 0.7, 1.3, 3.1],
               [0.6, 1.9, 0.0, 1.6, 2.2, 2.2, 1.6],
               [2.0, 3.5, 1.6, 0.0, 3.7, 3.6, 1.4],
               [1.7, 0.7, 2.2, 3.7, 0.0, 0.7, 3.1],
               [1.6, 1.3, 2.2, 3.6, 0.7, 0.0, 2.7],
               [1.5, 3.1, 1.6, 1.4, 3.1, 2.7, 0.0]]

In [3]:
items = [[1, 1000, 10, 0],
         [2, 200, 60 , 0], 
         [3, 300, 300 , 0], 
         [4, 400, 400, 0], 
         [5, 50, 4000, 0], 
         [6, 600, 200, 0], 
         [6, 700, 40 , 0]]

In [4]:
cr = 0.01
vvc = 12

In [5]:
total_time = 0
dt_minmax = []
n_minmax = []
n = []
visitday = {'Monday' : [], 'Tuesday' : [], 'Wednesday' : [], 'Thursday' : [], 'Friday' : []}
day_traveltime = {'Monday' : [], 'Tuesday' : [], 'Wednesday' : [], 'Thursday' : [], 'Friday' : []}
opt_traveltime = {'Monday' : [], 'Tuesday' : [], 'Wednesday' : [], 'Thursday' : [], 'Friday' : []}

# step2. Initialize

In [6]:
alpha = 0
opt_cost = math.inf

In [7]:
def cal_dt_minmax(vendor_dist):
    global dt_minmax
    for i in range(1,len(vendor_dist)):
        dtmax = vendor_dist[0][i] + vendor_dist[i][0]
        temp = vendor_dist[i][1:].copy()
        temp = [i for i in temp if i != 0]
        temp.sort()
        dtmin = temp[0] + temp[1]
        if dtmax <= dtmin:
            dtmax = dtmin
        dt_minmax.append([dtmin, dtmax])
        
    print('\ndtmin(k), dtmax(k) :\n', *dt_minmax, sep='\n')

In [8]:
def cal_n_minmax(vendor_dist,dt_minmax,cr,vvc):
    global n_minmax
    for i in range(1,len(vendor_dist)): 
        dc = 0
        sc = 0

        for j in items:
            if j[0] == i:
                dc += j[1]*j[2]
                sc += j[3]

        nmax = math.sqrt((0.5 * cr * dc)/(vvc*dt_minmax[i-1][0]))
        nmin = math.sqrt((0.5 * cr * dc)/(vvc*dt_minmax[i-1][1]))

        n_minmax.append([nmin, nmax])
        
    print('\nnmin(k), nmax(k) :\n', *n_minmax, sep='\n')    

In [9]:
cal_dt_minmax(vendor_dist)
cal_n_minmax(vendor_dist,dt_minmax,cr,vvc)


dtmin(k), dtmax(k) :

[2.0, 3.2]
[3.2, 3.2]
[3.0, 4.0]
[1.4, 3.4]
[2.0, 3.2]
[3.0, 3.0]

nmin(k), nmax(k) :

[1.141088661469096, 1.4433756729740645]
[1.25, 1.25]
[3.0618621784789726, 3.5355339059327378]
[4.428074427700476, 6.900655593423543]
[5.103103630798287, 6.454972243679028]
[4.533823502911814, 4.533823502911814]


# Step3. Calculate 'n'

In [10]:
def cal_n(n_minmax):
    global n
    global n_k
    
    for i in range(len(n_minmax)):
        picks = round(n_minmax[i][0] + (alpha*(n_minmax[i][1]-n_minmax[i][0])))
        if picks >= 6:
            picksnew = 5
        else:
            picksnew = picks
        n.append([picks,picksnew])
        n_k = deepcopy(n)
        
    print('\nn(k), Revised n(k) :\n', *n, sep = '\n')

In [11]:
cal_n(n_minmax)


n(k), Revised n(k) :

[1, 1]
[1, 1]
[3, 3]
[4, 4]
[5, 5]
[5, 5]


# Step4. Assign days

In [12]:
def assign_days(n):
    global visitday

    for i in range(len(n)):
        if n[i][1] == 1:
            visitday['Wednesday'].append(i+1)

        elif n[i][1] == 2:
            visitday['Tuesday'].append(i+1)
            visitday['Thursday'].append(i+1)

        elif n[i][1] == 3:
            visitday['Monday'].append(i+1)
            visitday['Wednesday'].append(i+1)
            visitday['Friday'].append(i+1)

        elif n[i][1] == 4:
            visitday['Monday'].append(i+1)
            visitday['Tuesday'].append(i+1)
            visitday['Thursday'].append(i+1)
            visitday['Friday'].append(i+1)

        elif n[i][1] == 5:
            visitday['Monday'].append(i+1)
            visitday['Tuesday'].append(i+1)
            visitday['Wednesday'].append(i+1)
            visitday['Thursday'].append(i+1)
            visitday['Friday'].append(i+1)
            

    

In [13]:
assign_days(n)

# Step5. Apply Clarke-Wright Heuristic

In [14]:
def cal_travel_time(visitday, vendor_dist):
    for key in visitday.keys():
        global day_traveltime

        clark = []
        total_dist = 0

        for i in visitday[key]:
            total_dist += 2 * vendor_dist[0][i]
            for j in visitday[key]:
                if i < j:
                    sij = vendor_dist[0][i] + vendor_dist[0][j] - vendor_dist[i][j]
                    clark.append([i,j, sij])

        day_traveltime[key].append(clark)
        day_traveltime[key].append(total_dist)
    
    #print('\nTime for Visit each Vendor (Individually):')
    #pprint.pprint(day_traveltime, sort_dicts = False)

In [15]:
cal_travel_time(visitday, vendor_dist)

In [16]:
def cal_opt_time(day_traveltime):
    print('\n****Clark-Wright****')
    for key in day_traveltime.keys():
        global opt_traveltime
        print('\n:::::::::::::',key,':::::::::::::::::::')
        max_start = 0
        connect = []

        clark = day_traveltime[key][0]
        total_dist = day_traveltime[key][1]
        
        while len(clark) != 0:
            maxsaving = 0
            #print('total dist', total_dist)

            for i in clark:
                if i[2] >= maxsaving:
                    maxsaving = i[2]
                    max_start = i[0]
                    temp = i
            connect.append([temp[0], temp[1]])
            #print('max saving', maxsaving)
            total_dist -= maxsaving


            for j in clark:
                clark = [i for i in clark if i[0] != max_start]
            #print('remainigs',clark)
            #print('\n')

        opt_traveltime[key].append(connect)
        opt_traveltime[key].append(total_dist)

        print('RESULT:',opt_traveltime[key])
    
    
    #print('\n****Clark-Wright(Finished)****')
    
    

In [17]:
cal_opt_time(day_traveltime)


****Clark-Wright****

::::::::::::: Monday :::::::::::::::::::
RESULT: [[[4, 5], [3, 6], [5, 6]], 8.500000000000002]

::::::::::::: Tuesday :::::::::::::::::::
RESULT: [[[4, 5], [5, 6]], 6.6]

::::::::::::: Wednesday :::::::::::::::::::
RESULT: [[[3, 6], [1, 5], [2, 3], [5, 6]], 9.200000000000001]

::::::::::::: Thursday :::::::::::::::::::
RESULT: [[[4, 5], [5, 6]], 6.6]

::::::::::::: Friday :::::::::::::::::::
RESULT: [[[4, 5], [3, 6], [5, 6]], 8.500000000000002]


# Visualize Clarke-Wright Heuristic schedule

In [18]:
def clack_sequence(k,i):
 
    global n
    n = 0
    temp = k.pop(0)
    if temp[0] == k[i][0]:
        temp = deque(temp)
        temp.appendleft(k[i][-1])
        del k[i]
        k = deque(k)
        k.appendleft(list(temp))
        k = list(k)
        n = 1
    elif temp[0] == k[i][-1]:
        temp = deque(temp)
        temp.appendleft(k[i][0])
        del k[i]
        k = deque(k)
        k.appendleft(list(temp))
        k = list(k)
        n = 1
    elif temp[-1] == k[i][0]:
        temp.append(k[i][-1])
        del k[i]
        k = deque(k)
        k.appendleft(temp)
        k = list(k)
        n =1
    elif temp[-1] == k[i][-1]:
        temp.append(k[i][0])
        del k[i]
        k = deque(k)
        k.appendleft(temp)
        k = list(k)
        n =1
    else:
        k = deque(k)
        k.appendleft(temp)
        k = list(k)
    
    return k

In [19]:
def make_sequence(k):
    while len(k) != 1:
        for i in range(len(k)-1):
            k = clack_sequence(k,i)
            if n == 1: 
                break
    
    return k

In [20]:
def make_schedule():
    global schedule
    global test_schedule
    global travel_time_day
    
    sequence_temp = []
    for key in day_traveltime.keys():
        sequence_temp.append(opt_traveltime[key][0])
    
    temp = []
    schedule = []
    for i in range(5):
        temp.append(make_sequence(sequence_temp[i]))
        schedule.append(temp[i][0])
    
    schedule_temp = deepcopy(schedule)
    
    test_schedule = []
    for i in range(len(schedule)):
        test_schedule.append(list(itertools.permutations(schedule_temp[i])))
    
    travel_time_day = []
    for key in day_traveltime.keys():
        travel_time_day.append(opt_traveltime[key][-1])

In [21]:
make_schedule()
print(f'Monday:{schedule[0]}')
print(f'Tuesday:{schedule[1]}')
print(f'Wednesday:{schedule[2]}')
print(f'Thursday:{schedule[3]}')
print(f'Friday:{schedule[4]}')

Monday:[4, 5, 6, 3]
Tuesday:[4, 5, 6]
Wednesday:[2, 3, 6, 5, 1]
Thursday:[4, 5, 6]
Friday:[4, 5, 6, 3]


# Step6. Pairwise exchange

In [22]:
def pairwise_cost_day(order):
    total_travel_time_test = 0
    travel_time_best = round(travel_time_day[order],7) #부동소수점 오류 해결
    new_order = 0
    
    for i in range(len(test_schedule[order])):
        temp = deque(list(test_schedule[order][i]))
        temp.appendleft(0)
        temp.append(0)
        
        for j in range(len(temp)-1):
            total_travel_time_test += vendor_dist[temp[j]][temp[j+1]]
            total_travel_time_test = round(total_travel_time_test,7)    #부동소수점 오류 해결
        if total_travel_time_test < travel_time_best:
            travel_time_best = total_travel_time_test
            new_order = temp
            print(test_schedule[order][i])
            print(travel_time_best)

        total_travel_time_test = 0
        
    
    return travel_time_best,new_order

In [23]:
def check_pairwise():
    new_order = 0
    week = ['월요일','화요일','수요일','목요일','금요일']
    global check
    check = 0
    for i in range(5):
        travel_time_day[i],new_order = pairwise_cost_day(i)
        if new_order != 0:
            print(f'{week[i]}의 vendor를 방문하는 순서가 바뀌었습니다')
            schedule[i] = list(new_order)
            check = 1
    return schedule

In [24]:
print(check_pairwise())
if check == 1:
    print('improvement occur')
else:
    print('no improvement')

[[4, 5, 6, 3], [4, 5, 6], [2, 3, 6, 5, 1], [4, 5, 6], [4, 5, 6, 3]]
no improvement


# Step 7. Calculate Total Time & Cost

In [25]:
def cal_total_cost(opt_traveltime, vvc):
    global total_time
    for key in opt_traveltime.keys():
        total_time += opt_traveltime[key][1]
    total_cost = total_time * vvc
    
    return total_cost

In [26]:
total_travel_cost = cal_total_cost(opt_traveltime, vvc)

In [27]:
p_k = [[1],[2],[3],[4],[5],[6,7]]

def cal_carrying_cost():
    global total_carrying_cost
    total_carrying_cost = 0
    for i in p_k:
        for j in i:
            total_carrying_cost += (0.5*0.01*items[j-1][1]*items[j-1][2])/n_k[i[0]-1][1]
    
    return total_carrying_cost

In [28]:
total_carrying_cost = cal_carrying_cost()

In [29]:
total_cost = total_carrying_cost + total_travel_cost

In [30]:
total_cost

1280.8000000000002

# Aggregate Code

In [31]:
vendor_dist = [[0.0, 1.6, 0.6, 2.0, 1.7, 1.6, 1.5],
               [1.6, 0.0, 1.9, 3.5, 0.7, 1.3, 3.1],
               [0.6, 1.9, 0.0, 1.6, 2.2, 2.2, 1.6],
               [2.0, 3.5, 1.6, 0.0, 3.7, 3.6, 1.4],
               [1.7, 0.7, 2.2, 3.7, 0.0, 0.7, 3.1],
               [1.6, 1.3, 2.2, 3.6, 0.7, 0.0, 2.7],
               [1.5, 3.1, 1.6, 1.4, 3.1, 2.7, 0.0]]

In [32]:
items = [[1, 1000, 10, 0],
         [2, 200, 60 , 0], 
         [3, 300, 300 , 0], 
         [4, 400, 400, 0], 
         [5, 50, 4000, 0], 
         [6, 600, 200, 0], 
         [6, 700, 40 , 0]]

In [37]:
alpha = 0
cr = 0.01
vvc = 12
opt_cost = math.inf
opt_alpha = 0

In [49]:
dt_minmax = []
n_minmax = []
cal_dt_minmax(vendor_dist)
cal_n_minmax(vendor_dist,dt_minmax,cr,vvc)

print('\n')

while alpha <= 1:
    print('Current alpha=',round(alpha,2),'----------------------------------------------------------------\n')
    total_time = 0
    n = []
    visitday = {'Monday' : [], 'Tuesday' : [], 'Wednesday' : [], 'Thursday' : [], 'Friday' : []}
    day_traveltime = {'Monday' : [], 'Tuesday' : [], 'Wednesday' : [], 'Thursday' : [], 'Friday' : []}
    opt_traveltime = {'Monday' : [], 'Tuesday' : [], 'Wednesday' : [], 'Thursday' : [], 'Friday' : []}

    cal_n(n_minmax)
    assign_days(n)
    cal_travel_time(visitday, vendor_dist)
    cal_opt_time(day_traveltime)
    
    print('\n<Sequence each day>')
    make_schedule()
    print(f'Monday:{schedule[0]}')
    print(f'Tuesday:{schedule[1]}')
    print(f'Wednesday:{schedule[2]}')
    print(f'Thursday:{schedule[3]}')
    print(f'Friday:{schedule[4]}')
    
    print('\n')
    print('After pairwise ')
    
    print(check_pairwise())
    if check == 1:
        print('improvement occur')
    else:
        print('no improvement')
    
    total_travel_cost = cal_total_cost(opt_traveltime, vvc)
    total_carrying_cost = cal_carrying_cost()
    total_cost = total_travel_cost + total_carrying_cost
    
    print('\nTOTAL COST:',total_cost)
    print('OPTIMUM COST:',opt_cost)
      
    if total_cost < opt_cost:
        opt_cost = total_cost
        opt_alpha = alpha
        print('Optimum is Changed\nNew:',opt_cost)
    else:
        print('Optimum is NOT Changed:',opt_cost)
    
    print('\n\n')
    alpha += 0.1

print('************************************************************************************************************\n')
print('Final Result:\nOptimum Cost:',opt_cost,'\nOptimum alpha:',round(opt_alpha,2))


dtmin(k), dtmax(k) :

[2.0, 3.2]
[3.2, 3.2]
[3.0, 4.0]
[1.4, 3.4]
[2.0, 3.2]
[3.0, 3.0]

nmin(k), nmax(k) :

[1.141088661469096, 1.4433756729740645]
[1.25, 1.25]
[3.0618621784789726, 3.5355339059327378]
[4.428074427700476, 6.900655593423543]
[5.103103630798287, 6.454972243679028]
[4.533823502911814, 4.533823502911814]


Current alpha= 0 ----------------------------------------------------------------


n(k), Revised n(k) :

[1, 1]
[1, 1]
[3, 3]
[4, 4]
[5, 5]
[5, 5]

****Clark-Wright****

::::::::::::: Monday :::::::::::::::::::
RESULT: [[[4, 5], [3, 6], [5, 6]], 8.500000000000002]

::::::::::::: Tuesday :::::::::::::::::::
RESULT: [[[4, 5], [5, 6]], 6.6]

::::::::::::: Wednesday :::::::::::::::::::
RESULT: [[[3, 6], [1, 5], [2, 3], [5, 6]], 9.200000000000001]

::::::::::::: Thursday :::::::::::::::::::
RESULT: [[[4, 5], [5, 6]], 6.6]

::::::::::::: Friday :::::::::::::::::::
RESULT: [[[4, 5], [3, 6], [5, 6]], 8.500000000000002]

<Sequence each day>
Monday:[4, 5, 6, 3]
Tuesday:[4, 5, 6

In [47]:
def vendorpickup(vendor_dist, items):
    dt_minmax = []
    n_minmax = []
    cal_dt_minmax(vendor_dist)
    cal_n_minmax(vendor_dist,dt_minmax,cr,vvc)

    print('\n')

    while alpha <= 1:
        print('Current alpha=',round(alpha,2),'----------------------------------------------------------------\n')
        total_time = 0
        n = []
        visitday = {'Monday' : [], 'Tuesday' : [], 'Wednesday' : [], 'Thursday' : [], 'Friday' : []}
        day_traveltime = {'Monday' : [], 'Tuesday' : [], 'Wednesday' : [], 'Thursday' : [], 'Friday' : []}
        opt_traveltime = {'Monday' : [], 'Tuesday' : [], 'Wednesday' : [], 'Thursday' : [], 'Friday' : []}

        cal_n(n_minmax)
        assign_days(n)
        cal_travel_time(visitday, vendor_dist)
        cal_opt_time(day_traveltime)
    
        print('\n<Sequence each day>')
        make_schedule()
        print(f'Monday:{schedule[0]}')
        print(f'Tuesday:{schedule[1]}')
        print(f'Wednesday:{schedule[2]}')
        print(f'Thursday:{schedule[3]}')
        print(f'Friday:{schedule[4]}')
    
        print('\n')
        print('After pairwise ')
    
        print(check_pairwise())
        if check == 1:
            print('improvement occur')
        else:
            print('no improvement')
    
        total_travel_cost = cal_total_cost(opt_traveltime, vvc)
        total_carrying_cost = cal_carrying_cost()
        total_cost = total_travel_cost + total_carrying_cost
    
        print('\nTOTAL COST:',total_cost)
        print('OPTIMUM COST:',opt_cost)
      
        if total_cost < opt_cost:
            opt_cost = total_cost
            opt_alpha = alpha
            print('Optimum is Changed\nNew:',opt_cost)
        else:
            print('Optimum is NOT Changed:',opt_cost)
    
        print('\n\n')
        alpha += 0.1

    print('************************************************************************************************************\n')
    print('Final Result:\nOptimum Cost:',opt_cost,'\nOptimum alpha:',round(opt_alpha,2))